In [ ]:
import lyncs_io as io
import numpy as np
import time

import matplotlib.pyplot as plt

import gluon_utils as gu
import gluon

In [ ]:
D_results = gluon.main(8,3)

In [ ]:
D_results = np.asarray(D_results)

In [ ]:
q = D_results[0,:]
D = D_results[1:,:]

In [ ]:
plt.scatter(q,q**2*D_results.mean(axis=0),s=0.5,c='k')
plt.yscale('log')
plt.xlabel(r'$qa$')
plt.ylabel(r'$q^2a^2D(q^2)$')